In [1]:
#Predicting E-Commerce Product Recommendation Ratings from Reviews

In [2]:
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix, classification_report

In [21]:
df=pd.read_csv('Womens Clothing E-Commerce Reviews.csv',keep_default_na=False)
df

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
...,...,...,...,...,...,...,...,...,...,...,...
23481,23481,1104,34,Great dress for many occasions,I was very happy to snag this dress at such a ...,5,1,0,General Petite,Dresses,Dresses
23482,23482,862,48,Wish it was made of cotton,"It reminds me of maternity clothes. soft, stre...",3,1,0,General Petite,Tops,Knits
23483,23483,1104,31,"Cute, but see through","This fit well, but the top was very see throug...",3,0,1,General Petite,Dresses,Dresses
23484,23484,1084,28,"Very cute dress, perfect for summer parties an...",I bought this dress for a wedding i have this ...,3,1,2,General,Dresses,Dresses


In [22]:
#Basic Data PreProcessing

In [23]:
df['Review']=(df['Title'].map(str)+' '+df['Review Text']).apply(lambda row: row.strip())
df['Rating']=[1 if rating>3 else 0 for rating in df['Rating']]
df=df[['Review','Rating']]
df

,Review,Rating
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,Some major design flaws I had such high hopes ...,0
3,"My favorite buy! I love, love, love this jumps...",1
4,Flattering shirt This shirt is very flattering...,1
...,...,...
23481,Great dress for many occasions I was very happ...,1
23482,Wish it was made of cotton It reminds me of ma...,0
23483,"Cute, but see through This fit well, but the t...",0
23484,"Very cute dress, perfect for summer parties an...",0


In [28]:
df=df[df['Review']!='']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22642 entries, 0 to 23485
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  22642 non-null  object
 1   Rating  22642 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 530.7+ KB


In [31]:
df['Rating'].value_counts()

1    17449
0     5193
Name: Rating, dtype: int64

In [33]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(df[['Review']],df['Rating'],random_state=42)
X_train.shape, X_test.shape

((16981, 1), (5661, 1))

In [38]:
y_train.shape,y_test.shape

((16981,), (5661,))

In [35]:
from collections import Counter
Counter(y_train),Counter(y_test)

(Counter({1: 13059, 0: 3922}), Counter({1: 4390, 0: 1271}))

In [39]:
y_train.value_counts()

1    13059
0     3922
Name: Rating, dtype: int64

In [40]:
#Experiment 1: Basic NLP Count Based Features
#WordCount
#CharacterCount
#Word Density
#PunctuationCount
#UppercaseCount
#TitleWordCount

#Ref: https://www.analyticsvidhya.com/blog/2018/04/a-comprehensive-guide-to-understand-and-implement-text-classification-in-python/

In [68]:
import string

X_train['char_count']=X_train['Review'].apply(len)
X_train['word_count']=X_train['Review'].apply(lambda x:len(x.split()))
X_train['word_density']=X_train['char_count']/(X_train['word_count']+1)
X_train['punctuation_count']=X_train['Review'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation)))
X_train['title_word_count']=X_train['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
X_train['upper_case_word_count']=X_train['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

X_test['char_count']=X_test['Review'].apply(len)
X_test['word_count']=X_test['Review'].apply(lambda x:len(x.split()))
X_test['word_density']=X_test['char_count']/(X_test['word_count']+1)
X_test['punctuation_count']=X_test['Review'].apply(lambda x: len("".join(_ for _ in x if _ in string.punctuation)))
X_test['title_word_count']=X_test['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.istitle()]))
X_test['upper_case_word_count']=X_test['Review'].apply(lambda x: len([wrd for wrd in x.split() if wrd.isupper()]))

In [61]:
X_train.head()

,Review,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count
12896,Soooo soft! This is a delightfully soft and fl...,268,52,5.056604,8,2,0
13183,"Had my eye on this, but dind't get I finally v...",399,84,4.694118,20,2,1
1496,I wanted to like this... I wanted to like this...,525,104,5.000000,19,2,2
5205,Beautiful blouse Bought this for my daughter i...,203,35,5.638889,10,2,0
13366,"Boxy. large. Boxy, unflattering, and large.\n\...",295,51,5.673077,22,2,0


In [62]:
#training a logistic regression model
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(solver='liblinear',C=1,random_state=42)

In [66]:
lr.fit(X_train.drop(['Review'],axis=1),y_train)

LogisticRegression(C=1, random_state=42, solver='liblinear')

In [70]:
pred=lr.predict(X_test.drop(['Review'],axis=1))

In [76]:
a=classification_report(y_test,pred)

C:\Users\ameen\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ameen\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ameen\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [77]:
print(a)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1271
           1       0.78      1.00      0.87      4390

    accuracy                           0.78      5661
   macro avg       0.39      0.50      0.44      5661
weighted avg       0.60      0.78      0.68      5661



In [81]:
pd.DataFrame(confusion_matrix(y_test,pred))

,0,1
0,0,1271
1,0,4390


In [82]:
#Precision: Out of all the predictions made by a model for a class, how many are acutally correct
#Recall: Out of all test data samples belonging to a class, how many was the model able to predict correctly

In [83]:
# Subjectivity quantifies the amount of personal opinion and factual information contained in the text. 
# The higher subjectivity means that the text contains personal opinion rather than factual information

In [93]:
import textblob
textblob.TextBlob('This is an AMAZING pair of Jeans!').sentiment

Sentiment(polarity=0.7500000000000001, subjectivity=0.9)

In [89]:
textblob.TextBlob('I really hated this UGLY T-shirt!').sentiment

Sentiment(polarity=-0.8875, subjectivity=0.85)

In [108]:
#Experiment 2: Features from Sentiment Analysis
X_train_snt_obj=X_train['Review'].apply(lambda row:textblob.TextBlob(row).sentiment)
X_train['Polarity']=[obj.polarity for obj in X_train_snt_obj.values]
X_train['Subjectivity']=[obj.subjectivity for obj in X_train_snt_obj.values]

X_test_snt_obj=X_test['Review'].apply(lambda row:textblob.TextBlob(row).sentiment)
X_test['Polarity']=[obj.polarity for obj in X_test_snt_obj.values]
X_test['Subjectivity']=[obj.subjectivity for obj in X_test_snt_obj.values]

In [107]:
a=X_train_snt_obj.values[0]
a

Sentiment(polarity=0.17045454545454547, subjectivity=0.49090909090909096)

In [106]:
a.subjectivity

0.49090909090909096

In [109]:
X_train.head()

,Review,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,Polarity,Subjectivity
12896,Soooo soft! This is a delightfully soft and fl...,268,52,5.056604,8,2,0,0.170455,0.490909
13183,"Had my eye on this, but dind't get I finally v...",399,84,4.694118,20,2,1,0.101944,0.719537
1496,I wanted to like this... I wanted to like this...,525,104,5.000000,19,2,2,0.186538,0.458761
5205,Beautiful blouse Bought this for my daughter i...,203,35,5.638889,10,2,0,0.625000,0.825000
13366,"Boxy. large. Boxy, unflattering, and large.\n\...",295,51,5.673077,22,2,0,0.329613,0.510268


In [110]:
X_test.head()

,Review,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,Polarity,Subjectivity
19566,"Overall great dress, runs small This dress is ...",531,94,5.589474,16,2,0,0.185179,0.638690
17836,"Very sheer Super flattering, a great length fo...",73,13,5.214286,6,2,0,0.283333,0.841667
9304,Staple cardigan I've been searching for the pe...,463,88,5.202247,10,1,0,0.219286,0.767619
4587,"The sales girl added this to my room, it's so ...",79,17,4.388889,3,1,0,0.000000,0.000000
14858,I bought the delaney blazer in size 6 -- and i...,465,90,5.109890,16,1,1,0.299167,0.435000


In [111]:
lr.fit(X_train.drop(['Review'],axis=1),y_train)
pred=lr.predict(X_test.drop(['Review'],axis=1))

In [112]:
a=classification_report(y_test,pred)

In [113]:
print(a)

              precision    recall  f1-score   support

           0       0.69      0.27      0.39      1271
           1       0.82      0.96      0.89      4390

    accuracy                           0.81      5661
   macro avg       0.75      0.62      0.64      5661
weighted avg       0.79      0.81      0.77      5661



In [114]:
pd.DataFrame(confusion_matrix(y_test,pred))

,0,1
0,342,929
1,157,4233


In [115]:
#Experiment 3: Text pre-processing and wrangling
#Using NLP BOW, ngrams, etc.

In [117]:
!pip install contractions
!pip install textsearch
!pip install tqdm
import nltk
nltk.download('punkt')
nltk.download('stopwords')

You should consider upgrading via the 'c:\users\ameen\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\ameen\anaconda3\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\ameen\anaconda3\python.exe -m pip install --upgrade pip' command.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ameen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ameen\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [118]:
import contractions
contractions.fix("I didn't like this t-shirt")

'I did not like this t-shirt'

In [122]:
import nltk
import contractions
import re

#remove some stopwords to capture negation in n-grams if possible
stop_words=nltk.corpus.stopwords.words('english')
stop_words.remove('no')
stop_words.remove('not')
stop_words.remove('but')

ps=nltk.porter.PorterStemmer()

def simple_text_preprocessor(document):
    #lower case
    document=str(document).lower()
    #expand contractions
    document=contractions.fix(document)
    #remove unneccessary characters
    document = re.sub(r'[^a-zA-Z]',r' ',document)
    document = re.sub(r'nbsp',r'',document)
    document = re.sub(' +',' ',document)
    #simple porter stemming
    document = ' '.join([ps.stem(word) for word in document.split()])
    #stopwords removal
    document = ' '.join([word for word in document.split() if word not in stop_words])
    
    return document
stp=np.vectorize(simple_text_preprocessor)

In [123]:
X_train['Clean Review']=stp(X_train['Review'].values)
X_test['Clean Review']=stp(X_test['Review'].values)

X_train.head()

,Review,char_count,word_count,word_density,punctuation_count,title_word_count,upper_case_word_count,Polarity,Subjectivity,Clean Review
12896,Soooo soft! This is a delightfully soft and fl...,268,52,5.056604,8,2,0,0.170455,0.490909,soooo soft thi delight soft fluffi sweater mig...
13183,"Had my eye on this, but dind't get I finally v...",399,84,4.694118,20,2,1,0.101944,0.719537,eye thi but dind get final visit store petit t...
1496,I wanted to like this... I wanted to like this...,525,104,5.000000,19,2,2,0.186538,0.458761,want like thi want like thi top badli badli fa...
5205,Beautiful blouse Bought this for my daughter i...,203,35,5.638889,10,2,0,0.625000,0.825000,beauti blous bought thi daughter law birthday ...
13366,"Boxy. large. Boxy, unflattering, and large.\n\...",295,51,5.673077,22,2,0,0.329613,0.510268,boxi larg boxi unflatt larg curvi pound thi to...
